** From Foundations.Lab3
- A workflow chat system

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [5]:
reader = PdfReader("../1_foundations/me/MK-profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

   
Contact
marko@blicqs.com
www.linkedin.com/in/marko-
kennedy (LinkedIn)
www.phroot.com (Company)
Top Skills
New Business Development
Information Security
New Business Development
Certifications
LFD125: Security for Software
Development Managers
LFD121: Developing Secure
Software
Certified Information Security
Manager (CISM)
LFS183: Introduction to Zero Trust
Certified Information Systems
Security Professional (CISSP)
Marko Kennedy
Security Architect at IRIS Software Group (CISSP, CISM)
Guernsey
Summary
Marko is a certified information systems security expert with key
software developer experience and skills. He is also a successful
Internet entrepreneur who created and sold a business to IRIS
Software Group (the UK’s largest private software business). He has
a valuable combination of higher level business skills and expert
technical competencies. 
Business specialties: InfoSec management, strategic planning,
operational management, start-up development, and exit planning
Technical 

In [8]:
with open("../1_foundations/me/MK-summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [9]:
name = "Marko Kennedy"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [11]:
system_prompt

"You are acting as Marko Kennedy. You are answering questions on Marko Kennedy's website, particularly questions related to Marko Kennedy's career, background, skills and experience. Your responsibility is to represent Marko Kennedy for interactions on the website as faithfully as possible. You are given a summary of Marko Kennedy's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Marko Kennedy. I founded three startups. One, CarCatcher.com, was a great lesson in how to do it better. I collect stamps and to compensate, I have a black belt in Karate.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nmarko@blicqs.com\nwww.linkedin.com/in/marko-\nkennedy (LinkedIn)\nwww.phroot.com (Company)\nTop Skills\nNew Business Development\nInformation Security\nNew Business Development\nCertifications\nLFD1

In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

openai = OpenAI()

This LLM call uses the "structured outputs" technique: we specify that the LLM must respond with an 
Evaluation object (ie. defined JSON format, created by the pydantic library).
The .parse(..., response_format=Evaluation) enforces this.

In [30]:
def evaluate(reply, message, history) -> Evaluation:
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    #response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [31]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [18]:
reply

'No, I do not hold a patent. My primary focus has been on developing startups and expanding my expertise in information security and software development. If you have any other questions about my background or experience, feel free to ask!'

In [32]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable as it directly answers the user's question about holding a patent with a clear 'No'. The Agent also provides additional context regarding their professional focus, which maintains an engaging tone. Moreover, the invitation for further questions fosters an interactive conversation, reflecting the professional and approachable demeanor required.")

In [35]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [34]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
        
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response "Iway odayn'tday avehay anay atentpay" appears to be a garbled form of Pig Latin rather than a clear, professional answer to the user's question. It's important for the Agent to maintain a professional demeanor and provide clear, understandable information. A better response would simply state that Marko does not hold any patents or to clarify if there are any related innovations or contributions.
